In [ ]:
🧠 What is Memory in LangChain?

Memory allows an LLM to remember past messages, facts, or context across multiple interactions.

Without memory:

User → LLM → Forget


With memory:

User → LLM → Remember → Use in next reply

🔥 Main Types of Memory in LangChain v1

-------------------------------------------
1️⃣ ConversationBufferMemory

Stores the entire conversation

Meaning

It keeps everything the user and AI said.

Example
User: Hi
AI: Hello
User: My name is Surbrmani
AI: Nice to meet you Subramani


The AI remembers “Subramani”.

Best for

Chatbots

Assistants

Customer support

------------------------------------------

2️⃣ ConversationBufferWindowMemory

Stores only the last N messages

Meaning

Only recent chat is remembered.

Example if window=2:

User: Hi   ❌ forgotten
User: My name is Subramani
AI: Hello Subramani
User: How are you?

Best for

Long chats

Cost control

Avoiding prompt overload

------------------------------------------

3️⃣ ConversationSummaryMemory

Stores a summary instead of full chat

Meaning

LLM compresses conversation into a short summary.

Example

"User is Subramani and wants to learn LangChain"

Best for

Long-running assistants

Bots that chat for hours

Production systems

------------------------------------------

4️⃣ ConversationSummaryBufferMemory

Keeps recent messages + summary

Meaning

Best of both worlds:

Latest chat verbatim

Old chat summarized

Best for

Chatbots

Agents

RAG assistants

------------------------------------------

5️⃣ VectorStoreMemory

Stores memory in a vector database

Meaning

Past chats are embedded and searched when needed.

Like:

“Find things the user said before that are relevant now”

Best for

Long-term memory

Personal AI assistants

Customer profiles

------------------------------------------

6️⃣ EntityMemory

Remembers important facts about user

Example:

User: My company is IBM
User: I live in Coimbatore


Stored as:

Company = IBM
Location = Coimbatore

Best for

CRM bots

Sales assistants

Personalized AI

🧠 One-Table Summary
| Memory Type      | What it remembers           |
| ---------------- | --------------------------- |
| Buffer           | Full chat                   |
| Window           | Last few messages           |
| Summary          | Short summary               |
| Summary + Buffer | Best of both                |
| VectorStore      | Long-term searchable memory |
| Entity           | User facts                  |

🧠 LangChain v1 Memory → Database View
| Memory Type          | What it remembers           | Where it is stored                                  |
| -------------------- | --------------------------- | --------------------------------------------------- |
| **Buffer**           | Full chat                   | RAM (Python list)                                   |
| **Window**           | Last N messages             | RAM (sliding list)                                  |
| **Summary**          | Compressed chat             | RAM (string summary)                                |
| **Summary + Buffer** | Summary + recent chat       | RAM                                                 |
| **VectorStore**      | Long-term searchable memory | **Vector Database** (FAISS, Chroma, Pinecone, etc.) |
| **Entity**           | User facts                  | **Key–Value Store** (dict / Redis / DB)             |
 

In [12]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

load_dotenv(".env")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}")
])

base_chain = prompt | llm



In [11]:
#1️⃣ Buffer Memory – remembers everything
# Full chat history
store = {}

def get_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

chat = RunnableWithMessageHistory(
    base_chain,
    get_history,
    input_messages_key="input",
)

print(chat.invoke({"input": "My name is Superman"}, config={"configurable": {"session_id": "u1"}}).content)
print(chat.invoke({"input": "What is my name?"}, config={"configurable": {"session_id": "u1"}}).content)


Hello, Superman! How can I assist you today?
Your name is Superman! How can I assist you further?


In [13]:
#2️⃣ Window Memory – only last few messages
"""
Only last 2 user+AI messages are kept.
Older ones are forgotten.
"""

from collections import deque

store = {}

def get_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory(max_messages=4)  # 2 user + 2 AI
    return store[session_id]

chat = RunnableWithMessageHistory(
    base_chain,
    get_history,
    input_messages_key="input",
)

print(chat.invoke({"input": "Hi"}, config={"configurable": {"session_id": "u2"}}).content)
print(chat.invoke({"input": "My name is Superman"}, config={"configurable": {"session_id": "u2"}}).content)
print(chat.invoke({"input": "What is my name?"}, config={"configurable": {"session_id": "u2"}}).content)




Hello! How can I assist you today?
Hello, Superman! It's great to meet you. How can I assist you today?
Your name is Superman! How can I assist you today?


In [14]:
#3️⃣ Summary Memory – Remembers a summary
"""
Old messages are summarized instead of stored.
"""

summary_store = {}

def get_history(session_id):
    if session_id not in summary_store:
        summary_store[session_id] = ChatMessageHistory()
    return summary_store[session_id]

chat = RunnableWithMessageHistory(
    base_chain,
    get_history,
    input_messages_key="input",
)

print(chat.invoke({"input": "My name is Superman"}, config={"configurable": {"session_id": "u3"}}).content)
print(chat.invoke({"input": "I am learning LangChain"}, config={"configurable": {"session_id": "u3"}}).content)

# Normally LangChain would summarize old messages here
print(chat.invoke({"input": "Who am I?"}, config={"configurable": {"session_id": "u3"}}).content)


Hello, Superman! How can I assist you today?
That's great to hear, Superman! LangChain is a powerful framework for building applications with language models. What specific aspects of LangChain are you learning about, or do you have any questions regarding it?
You are Superman! How can I assist you further today?


In [15]:
#4️⃣ Summary + Buffer – Best of both
"""
Recent messages kept.
Older messages summarized.
"""

summary = "User is Superman"
store = {}

def get_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

chat = RunnableWithMessageHistory(
    base_chain,
    get_history,
    input_messages_key="input",
)

print(chat.invoke({"input": "My name is Superman"}, config={"configurable": {"session_id": "u4"}}).content)
print(chat.invoke({"input": "What is my name?"}, config={"configurable": {"session_id": "u4"}}).content)


Hello, Superman! How can I assist you today?
Your name is Superman! How can I assist you further?


In [ ]:
#5️⃣ VectorStore Memory – Long-term searchable memory
"""
Chat history stored in vector DB.
AI retrieves relevant old info.
"""

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts([], embeddings)
retriever = vectorstore.as_retriever()

# Store memory
vectorstore.add_texts(["My company is IBM", "I live in Coimbatore"])

# Retrieve memory
docs = retriever.get_relevant_documents("Where do I work?")
print(docs[0].page_content)


In [18]:
#6️⃣ Entity Memory – User facts
"""
Stores structured user facts like a database
"""

entity_store = {}

entity_store["name"] = "Superman"
entity_store["company"] = "IBM"
entity_store["city"] = "Coimbatore"

print("Name:", entity_store["name"])
print("Company:", entity_store["company"])
print("City:", entity_store["city"])


Name: Superman
Company: IBM
City: Coimbatore
